In [1]:
import pandas as pd
import numpy as np
from viet_text_tools import normalize_diacritics
from pyvi import ViTokenizer
from underthesea import word_tokenize
import re

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer, TfidfTransformer

ModuleNotFoundError: No module named 'viet_text_tools'

# Food description

In [ ]:
from pathlib import Path
path = Path(__file__).parent.parent

In [6]:
df = pd.read_excel(path+"\Food_description.xlsx")

In [7]:
df.columns

Index(['FoodID', 'Tên món ăn', 'Vùng miền', 'Thành phần',
       'Thể loại (nướng, nước, khô, ...)', 'mô tả ngắn gọn ( <100 từ)',
       'Unnamed: 6'],
      dtype='object')

In [8]:
df['Vùng miền'].value_counts()

Miền Bắc, Miền Trung, Miền Nam     10
Miền Nam                            6
Miền Trung                          5
Miền Bắc                            4
Miền Trung, Miền Nam                2
Khắp cả nước                        1
Miền Trung, Miền Bắc                1
Miền Bắc , Miền Trung, Miền Nam     1
Name: Vùng miền, dtype: int64

In [10]:
df['is_north'] = 0
df['is_south'] = 0
df['is_middle'] = 0

for i in range(df.shape[0]):
  if 'Miền Bắc' in str(df.loc[i, 'Vùng miền']):
    df.loc[i, 'is_north'] = 1
  if 'Miền Trung' in str(df.loc[i, 'Vùng miền']):
    df.loc[i, 'is_middle'] = 1
  if 'Miền Nam' in str(df.loc[i, 'Vùng miền']):
    df.loc[i, 'is_south'] = 1
  if "Khắp cả nước" in str(df.loc[i, 'Vùng miền']):
    df.loc[i, 'is_north'] = 1
    df.loc[i, 'is_south'] = 1
    df.loc[i, 'is_middle'] = 1
  

In [11]:
df['is_mon_kho'] = 0
df['is_mon_nuoc'] = 0
df['is_mon_nuong'] = 0

for i in range(df.shape[0]):
  if "Món khô" in str(df.loc[i, 'Thể loại (nướng, nước, khô, ...)']):
    df.loc[i, 'is_mon_kho'] = 1
  if "Khô" in str(df.loc[i, 'Thể loại (nướng, nước, khô, ...)']):
    df.loc[i, 'is_mon_kho'] = 1
  if "Món nước" in str(df.loc[i, 'Thể loại (nướng, nước, khô, ...)']):
    df.loc[i, 'is_mon_nuoc'] = 1
  if "Nước" in str(df.loc[i, 'Thể loại (nướng, nước, khô, ...)']):
    df.loc[i, 'is_mon_nuoc'] = 1
  if 'Nướng' in str(df.loc[i, 'Thể loại (nướng, nước, khô, ...)']):
    df.loc[i, 'is_mon_nuong'] = 1

In [12]:
if 'Nước' or "Món nước"  in str(df.loc[0, 'Thể loại (nướng, nước, khô, ...)']):
    print(True)

True


In [46]:
df.iloc[:, [0, 6, 7, 8, 9, 10, 11] ].to_csv("/content/drive/MyDrive/Food Recommend Project/data/food_features.csv", index = None)

In [47]:
df["description"] = df['Thành phần']+" "+df['mô tả ngắn gọn ( <100 từ)']

In [53]:
def get_stopwords_list(stop_file_path):
    """load stop words """
    
    with open(stop_file_path, 'r', encoding="utf-8") as f:
        stopwords = f.readlines()
        stop_set = set(m.strip() for m in stopwords)
        return list(frozenset(stop_set))
stopwords_path = "/content/drive/MyDrive/Food Recommend Project/data/vietnamese.txt"
stopwords = get_stopwords_list(stopwords_path)

In [ ]:
stopwords

In [55]:
def description_processing(text):
  text = text.lower()

  html = re.compile("<.*?>")
  text = html.sub(r"", text)

  text = re.sub("[^\w\s]", "", text)

  text = normalize_diacritics(text)

  text = word_tokenize(text, format = 'text')

  text = text.split(" ")
  text = [w for w in text if not w.lower() in stopwords]
  text = ' '.join(text)

  
  return text

In [62]:
processed_description = []
for i in range(df.shape[0]):
  processed_description.append(description_processing(df.loc[i, 'description']))

In [64]:
df['processed_description'] = processed_description
df[['processed_description', 'description']]

,processed_description,description
0,gạo nếp đậu xanh vừng đậu phộng xôi gạo nếp hấ...,"Gạo nếp, đậu xanh, Vừng, Đậu phộng Xôi được là..."
1,mìnước lèo thịt tôm cá trứng đậu phộng bánh ...,"mì,nước lèo, thịt, tôm, cá, trứng , đậu phộng..."
2,bánh phở lèo thịt bò trứng chần hành lá tương ...,"bánh phở, nước lèo, thịt bò, trứng chần, hành ..."
3,gạo thịt sườn heo mỡ hành trứng chiên rau chua...,"Gạo tấm, thịt sườn heo, mỡ hành, trứng chiên, ..."
4,miến dong thịt gà nấm rau thơm miến gà vị thịt...,"Miến dong, thịt gà, nấm, rau thơm Miến gà có v..."
5,bánh phở lèo thịt gà hành lá ớt món nổi tiếng ...,"bánh phở, nước lèo, thịt gà, hành lá, ớt Món ă..."
6,bún đậu chả cốm nem dưa leo rau thơm mắm tôm m...,"Bún, đậu, chả cốm, nem, lòng, dưa leo, rau thơ..."
7,bún thịt bò thịt heo huyết bò dầu hành lá sả ớ...,"Bún, thịt bò, thịt heo, huyết bò, dầu điều, hà..."
8,bún riêu cua thịt heo đậu cà chua huyết heo ch...,"Bún, riêu cua, thịt heo, đậu, cà chua, huyết h..."
9,bánh mì thịt heo rau thơm đồ chua pate ớt bánh...,"Bánh mì, thịt heo, rau thơm, đồ chua, pate, ớt..."


# Create TF-IDF matrix

In [70]:

vectorizer = TfidfVectorizer()
vectorizer = vectorizer.fit(df['processed_description'])
tf_idf_matrix = vectorizer.transform(df['processed_description'])


In [74]:
numpy_matrix = tf_idf_matrix.toarray()
numpy_matrix.shape

(30, 393)

In [76]:
idf_matrix = pd.DataFrame(numpy_matrix)
idf_matrix['FoodID'] = df['FoodID']

In [78]:
idf_matrix.to_csv("/content/drive/MyDrive/Food Recommend Project/data/food_description_idf.csv", index = None)

# Create Embedding matrix

In [81]:
df[['FoodID', 'processed_description']].to_csv("/content/drive/MyDrive/Food Recommend Project/data/food_description.csv", index = None)